In [1]:
import numpy as np
import pandas as pd
import os, sys

In [2]:
orig_dir = "C:/Users/thebr/Desktop/Programming/melodic-classifier/testing_data"

def getFiles(path, substring, *args):
    array = []
    for root, dirs, files in os.walk(path):
            for file in files:
                stop = False
                if substring in file:
                    for arg in args: 
                        if arg in file: 
                            stop = True
                    if stop is True: 
                        continue
                    filepath = root + '/' + file
                    array.append(filepath)
                    # print(filepath)
    return array;

In [3]:
os.chdir(orig_dir)

f_artifact = []
t_artifact = []
f_non_artifact = []
t_non_artifact = []
count = 0
for root, dirs, files in os.walk(orig_dir):
    for directory in dirs:
        # make this an option later
        if "Preclin" in directory:
            os.chdir(directory)
            # the files that indicate which id's are artifacts
            artifact_numbers_file = open("fix_s8.txt")
            line = artifact_numbers_file.readline()
            artifact_numbers_file.close()
            artifact_numbers = line.strip().split(",")
            # every var after "t" is substrings to exclude
            f_files = getFiles(root + '/' + directory, "f", "EV", "index", "IC", "._", ".html", "DS", "png", "fix")
            t_files = getFiles(root + '/' + directory, "t", "EV", "index", "IC", "._", "f", ".html", "DS", "png", "fix")
            for file_path in f_files:
                filename = file_path.split("/")[-1]
                file_id = filename.split("f")[1]
                file_id = file_id.split(".")[0]
                if file_id in artifact_numbers:
                    f_artifact.append(file_path)
                else:
                    f_non_artifact.append(file_path)
            for file_path in t_files:
                filename = file_path.split("/")[-1]
                file_id = filename.split("t")[1]
                file_id = file_id.split(".")[0]
                if file_id in artifact_numbers:
                    t_artifact.append(file_path)
                else:
                    t_non_artifact.append(file_path)
        os.chdir(root)
        


In [7]:
from sklearn.model_selection import train_test_split
from numpy import median

# load all the text file data
# we're loading both the f and t file data, 
# the first half of the matrix will be the 
# f data and the second half the t data
data = []
labels = []
count = 0 
for f in f_artifact: 
    file = open(f)
    temp = []
    count = 0
    for line in file: 
        # Fill values after 88 with median
        if count == 88: 
            median = np.median(temp)
            while count < 175: 
                for i in range(0, 175 - 88): 
                    temp.append(median)
                    data.append(temp)
                count += 1
        # Get the current file's numbers as one line
        temp.append(float(line.strip()))
        count += 1
    data.append(temp)
    labels.append(1)

for f in f_non_artifact: 
    file = open(f)
    temp = []
    count = 0
    for line in file: 
        # Fill values after 88 with median
        if count == 88: 
            median = np.median(temp)
            while count < 175: 
                for i in range(0, 175 - 88): 
                    temp.append(median)
                    data.append(temp)
                count += 1
        # Get the current file's numbers as one line
        temp.append(float(line.strip()))
        count += 1
    data.append(temp)
    labels.append(0)
    
for t in t_artifact: 
    file = open(t)
    temp = []
    count = 0
    for line in file: 
        # Clip all values after 175
        if count == 175: 
            break
        # Get the current file's numbers as one line
        temp.append(float(line.strip()))
        count += 1
    data.append(temp)
    labels.append(1)

for t in t_non_artifact: 
    file = open(t)
    temp = []
    count = 0
    for line in file: 
        # Clip all values after 175
        if count == 175: 
            break
        # Get the current file's numbers as one line
        temp.append(float(line.strip()))
        count += 1
    data.append(temp)
    labels.append(0)

print(len(data))
# for i in data: 
#     print(len(i))
#     if len(data) != 175: 
#         print(i)
# print(type(data[0][0]))
# data = np.array(data)
# labels = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

478085


ValueError: Found input variables with inconsistent numbers of samples: [478085, 1238]

In [5]:
# for i in range(0, len(data[0])): 
#     print(data[0][i])
# print(len(data[0]))

In [6]:
from sklearn.preprocessing import Imputer

imputer = Imputer(strategy="median")

imputer.fit_transform(X_train)

C:\Users\thebr\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


NameError: name 'X_train' is not defined

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

# print(X_train.shape)
# print(y_train.shape)

# svm_clf = SVC(kernel="poly", degree=3, coef0=1, C=5)
# svm_clf.fit(X_train, y_train)
# y_pred = svm_clf.predict(X_test)

accuracy_score(y_test, y_pred)

# score = cross_val_score(log_clf, X_train, y_train, cv=5, verbose=3)
# score.mean()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=5)
rnd_search_cv.fit(X_train, y_train)

In [ ]:
rnd_search_cv.best_estimator_

In [ ]:
rnd_search_cv.best_score_

In [ ]:
rnd_search_cv.best_estimator_.fit(X_train, y_train)

y_pred = rnd_search_cv.best_estimator_.predict(X_test)
accuracy_score(y_test, y_pred)